EnrolledCSV

In [78]:
import pandas as pd
import numpy as np
import glob

enrolment_files = glob.glob("InputEnrolled/api_data_aadhar_enrolment_*.csv")

enrolment_df = pd.concat(
    [pd.read_csv(f) for f in enrolment_files],
    ignore_index=True
)

print("Files loaded:", len(enrolment_files))
print("Shape:", enrolment_df.shape)

enrolment_df.head()

Files loaded: 3
Shape: (1006029, 7)


,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,31-12-2025,Karnataka,Bidar,585330,2,3,0
1,31-12-2025,Karnataka,Bidar,585402,6,0,0
2,31-12-2025,Karnataka,Bidar,585413,1,0,0
3,31-12-2025,Karnataka,Bidar,585418,1,2,0
4,31-12-2025,Karnataka,Bidar,585421,4,3,0


In [79]:
demographic_files = glob.glob("InputDemographic/api_data_aadhar_demographic_*.csv")

demographic_df = pd.concat(
    [pd.read_csv(f) for f in demographic_files],
    ignore_index=True
)

print("Files loaded:", len(demographic_files))
print("Shape:", demographic_df.shape)

demographic_df.head()

Files loaded: 5
Shape: (2071700, 6)


,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,18-11-2025,Karnataka,Hasan,573118,0,2
1,18-11-2025,Karnataka,Hasan,573124,1,3
2,18-11-2025,Karnataka,Hasan,573150,0,2
3,18-11-2025,Karnataka,Hassan,573113,0,1
4,18-11-2025,Karnataka,Hassan,573120,1,4


In [80]:
biometric_files = glob.glob("InputBiometricupdate/api_data_aadhar_biometric_*.csv")

biometric_df = pd.concat(
    [pd.read_csv(f) for f in biometric_files],
    ignore_index=True
)

print("Files loaded:", len(biometric_files))
print("Shape:", biometric_df.shape)

biometric_df.head()

Files loaded: 4
Shape: (1861108, 6)


,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,19-09-2025,Goa,North Goa,403502,0,4
1,19-09-2025,Goa,North Goa,403508,1,4
2,19-09-2025,Goa,North Goa,403513,2,0
3,19-09-2025,Goa,North Goa,403527,2,2
4,19-09-2025,Goa,South Goa,403601,7,3


In [81]:
# keep only rows where state and district are NOT numeric
enrolment_df_copy = (enrolment_df[
    ~pd.to_numeric(enrolment_df["state"], errors="coerce").notna() &
    ~pd.to_numeric(enrolment_df["district"], errors="coerce").notna()
]).copy()

enrolment_df_copy.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,31-12-2025,Karnataka,Bidar,585330,2,3,0
1,31-12-2025,Karnataka,Bidar,585402,6,0,0
2,31-12-2025,Karnataka,Bidar,585413,1,0,0
3,31-12-2025,Karnataka,Bidar,585418,1,2,0
4,31-12-2025,Karnataka,Bidar,585421,4,3,0


In [82]:
enrolment_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1006007 entries, 0 to 1006028
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   date            1006007 non-null  object
 1   state           1006007 non-null  object
 2   district        1006007 non-null  object
 3   pincode         1006007 non-null  int64 
 4   age_0_5         1006007 non-null  int64 
 5   age_5_17        1006007 non-null  int64 
 6   age_18_greater  1006007 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 61.4+ MB


✅ TABLE 1 — Enrolment presence table

Columns should be:

state | district | pincode | has_enrolment_5_17

In [83]:
groupcolumns = ["state","district","pincode"]
Enrolment_presence_table = (enrolment_df_copy
                            .groupby(groupcolumns)
                            .agg(has_enrolment_5_17=("age_5_17",lambda x: (x>0).any()))
                            .reset_index())

In [84]:
Enrolment_presence_table.head()

,state,district,pincode,has_enrolment_5_17
0,Andaman & Nicobar Islands,Andamans,744101,True
1,Andaman & Nicobar Islands,Andamans,744103,True
2,Andaman & Nicobar Islands,Andamans,744105,False
3,Andaman & Nicobar Islands,Andamans,744106,True
4,Andaman & Nicobar Islands,Andamans,744107,True


✅ TABLE 2 — Enrolment presence table

Columns should be:

state | district | pincode | has_enrolment_5_17

In [85]:
# keep only rows where state and district are NOT numeric
demographic_df_copy = (demographic_df[
    ~pd.to_numeric(demographic_df["state"], errors="coerce").notna() &
    ~pd.to_numeric(demographic_df["district"], errors="coerce").notna()
]).copy()

In [86]:
demographic_presence_table = (demographic_df_copy
                              .groupby(groupcolumns)
                              .agg(has_demographic_5_17=("demo_age_5_17",lambda x: (x>0).any()))
                              .reset_index())

In [87]:
demographic_presence_table.head()

,state,district,pincode,has_demographic_5_17
0,Andaman & Nicobar Islands,Andamans,744101,True
1,Andaman & Nicobar Islands,Andamans,744102,False
2,Andaman & Nicobar Islands,Andamans,744103,True
3,Andaman & Nicobar Islands,Andamans,744105,True
4,Andaman & Nicobar Islands,Andamans,744106,False


✅ TABLE 3 — Biometric updates presence table

Columns should be:

state | district | pincode | has_enrolment_5_17

In [88]:
# keep only rows where state and district are NOT numeric
biometric_df_copy = (biometric_df[
    ~pd.to_numeric(biometric_df["state"], errors="coerce").notna() &
    ~pd.to_numeric(biometric_df["district"], errors="coerce").notna()
]).copy()

In [89]:
biometric_presence_table = (biometric_df_copy
                            .groupby(groupcolumns)
                            .agg(has_biometric_5_17=("bio_age_5_17",lambda x: (x>0).any()))
                            .reset_index())

In [90]:
biometric_presence_table.head()

,state,district,pincode,has_biometric_5_17
0,Andaman & Nicobar Islands,Andamans,744101,True
1,Andaman & Nicobar Islands,Andamans,744102,False
2,Andaman & Nicobar Islands,Andamans,744103,True
3,Andaman & Nicobar Islands,Andamans,744105,True
4,Andaman & Nicobar Islands,Andamans,744106,True


In [91]:
pin_presence = Enrolment_presence_table.merge(
    demographic_presence_table,
    on=["state","district","pincode"],
    how="outer")

In [92]:
pin_presence = pin_presence.merge(
    biometric_presence_table,
    on=["state","district","pincode"],
    how="outer")

In [93]:
pin_presence = pin_presence.fillna(False)

/var/folders/wr/w1_zrsw175n749bp4qzldbw00000gn/T/ipykernel_1176/756919168.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pin_presence = pin_presence.fillna(False)


In [95]:
pin_presence.head()

,state,district,pincode,has_enrolment_5_17,has_demographic_5_17,has_biometric_5_17
0,Andaman & Nicobar Islands,Andamans,744101,True,True,True
1,Andaman & Nicobar Islands,Andamans,744102,False,False,False
2,Andaman & Nicobar Islands,Andamans,744103,True,True,True
3,Andaman & Nicobar Islands,Andamans,744105,False,True,True
4,Andaman & Nicobar Islands,Andamans,744106,True,False,True


🔒 FINAL, LOCKED SET (with your change)
Enrol	Update	Biometric	lifecycle_state
T	T	T	Complete
T	T	F	No Biometric
T	F	F	Only Enrolment
F	T	F	Only Update
F	F	F	No Service
Other combo			Unclear

In [98]:
conditions = [((pin_presence["has_enrolment_5_17"] == True) & (pin_presence["has_demographic_5_17"] == True) & (pin_presence["has_biometric_5_17"] == True)),
              ((pin_presence["has_enrolment_5_17"] == True) & (pin_presence["has_demographic_5_17"] == True) & (pin_presence["has_biometric_5_17"] == False)),
              ((pin_presence["has_enrolment_5_17"] == True) & (pin_presence["has_demographic_5_17"] == False) & (pin_presence["has_biometric_5_17"] == False)),
              ((pin_presence["has_enrolment_5_17"] == False) & (pin_presence["has_demographic_5_17"] == True) & (pin_presence["has_biometric_5_17"] == False)),
              ((pin_presence["has_enrolment_5_17"] == False) & (pin_presence["has_demographic_5_17"] == False) & (pin_presence["has_biometric_5_17"] == False))]

lifecycle_state_names = ['Complete','No Biometric','Only Enrolment','Only Demographic','No Activity']

In [99]:
pin_presence["lifecycle_state"] = np.select(condlist=conditions,choicelist=lifecycle_state_names,default='Unclear')

### Pincode wise lifecycle states

In [101]:
pin_presence.head()

,state,district,pincode,has_enrolment_5_17,has_demographic_5_17,has_biometric_5_17,lifecycle_state
0,Andaman & Nicobar Islands,Andamans,744101,True,True,True,Complete
1,Andaman & Nicobar Islands,Andamans,744102,False,False,False,No Activity
2,Andaman & Nicobar Islands,Andamans,744103,True,True,True,Complete
3,Andaman & Nicobar Islands,Andamans,744105,False,True,True,Unclear
4,Andaman & Nicobar Islands,Andamans,744106,True,False,True,Unclear


In [105]:
pin_presence["lifecycle_state"].value_counts()

lifecycle_state
Complete            23844
Unclear              5521
No Activity          2634
Only Enrolment        483
Only Demographic      447
No Biometric           96
Name: count, dtype: int64

In [107]:
District_level_lifecycle = (pin_presence
                            .groupby(["state","district","lifecycle_state"])
                            .agg(pincode_count=("lifecycle_state","size"))
                            .reset_index())

In [109]:
District_level_lifecycle.head(50)

,state,district,lifecycle_state,pincode_count
0,Andaman & Nicobar Islands,Andamans,Complete,3
1,Andaman & Nicobar Islands,Andamans,No Activity,5
2,Andaman & Nicobar Islands,Andamans,Unclear,4
3,Andaman & Nicobar Islands,Nicobars,Unclear,1
4,Andaman & Nicobar Islands,South Andaman,No Activity,2
5,Andaman & Nicobar Islands,South Andaman,Unclear,4
6,Andaman and Nicobar Islands,Nicobar,Complete,1
7,Andaman and Nicobar Islands,Nicobar,Unclear,3
8,Andaman and Nicobar Islands,North And Middle Andaman,Complete,2
9,Andaman and Nicobar Islands,North And Middle Andaman,No Activity,1
